In [ ]:
# 1. Write your custom function here!

# For this demo, we do a basic NDVI calculation, inputting and returning a pandas DataFrame.
# Note that the function must input and return a pandas DataFrame. This is a requirement.
def compute_ndvi(df):
    # Perform your calculations
    df['ndvi'] = (df['SR_B5'] - df['SR_B4']) / (df['SR_B5'] + df['SR_B4'])
    return df

In [ ]:
# 2. Authenticate to Google Earth Engine, if you haven't done so already on your machine!
import ee

ee.Authenticate()

In [4]:
# 3. Run some Earth Engine code! This will do a basic grab of some Landsat data, with some standard filtering. 
import ee
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

# Basic cloud masking algorithm
def prep_sr_l8(image):
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Cirrus
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow
    qa_mask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturation_mask = image.select('QA_RADSAT').eq(0)

    # Apply the scaling factors to the appropriate bands.
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    # Replace the original bands with the scaled ones and apply the masks.
    return (image.addBands(optical_bands, None, True)
                 .addBands(thermal_bands, None, True)
                 .updateMask(qa_mask)
                 .updateMask(saturation_mask))

Plumas_Boundaries = ee.FeatureCollection("projects/robust-raster/assets/boundaries/Plumas_National_forest")
ic = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate('2018-05-08', '2018-05-10').map(prep_sr_l8).select(['SR_B4', 'SR_B5'])

In [ ]:
# 5. Preview the dataset and the results before doing a full run!
# This allows users to inspect the structure and content of the data to ensure it behaves as expected prior to running a full computation.
from robustraster import run

run(
    dataset=ic,
    source="ee",
    dataset_kwargs={
    'geometry': Plumas_Boundaries,
    'crs': 'EPSG:3310',
    'scale': 30
},
    user_function=compute_ndvi,
    preview_dataset=True
)

In [ ]:
# 6. Do the full computation here!
# Click on the link to the Dask dashboard to view the computation in real time!
run(
    dataset=ic,
    source="ee",
    dataset_kwargs={
    'geometry': Plumas_Boundaries,
    'crs': 'EPSG:3310',
    'scale': 30
},
    user_function=compute_ndvi,
    export_kwargs={
        "flag": "GTiff", 
        "output_folder": "tiles33", 
        "vrt": True},
    dask_mode="custom",
    dask_kwargs={
        "n_workers": 4,
        "threads_per_worker": 1,
        "memory_limit": "1GB"
    }
)